# 使用自定义Image进行推理

需要用到Tensorflow 和Keras , kernel 选择conda_tensorflow_p36

## 把训练好的模型存放到指定路径下
`output/model/tf_server/`

目录结构如下
```
└── tf_server
    └── 1
        ├── saved_model.pb
        └── variables
            ├── variables.data-00000-of-00001
            └── variables.index
```

## Build容器
对外提供服务时，需要使用容器方式。  
图片分类本身可不使用自定义容器，这里只是演示如何自定义容器。

In [1]:
%%sh
docker build -t sagemaker-cat-vs-dog .

Sending build context to Docker daemon  433.9MB
Step 1/5 : FROM tensorflow/serving
 ---> e0fe79fbb64f
Step 2/5 : RUN apt-get update && apt-get install -y --no-install-recommends nginx git
 ---> Running in 656fd3dd8500
Get:1 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [116 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [901 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic/restricted amd64 Packages [13.5 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic/multiverse amd64 Packages [186 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic/universe amd64 Packages [11.3 MB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 

## 本地运行docker

In [2]:
%%sh
docker run -p 8501:8501 -d sagemaker-cat-vs-dog

36ca45ff7124fd153660d4e96c0f6ed4e0330588f4bd72e60305913a625d6345


## 用本地Docker测试

In [3]:
import requests
import numpy as np
import json
import boto3 
IMAGE_WIDTH = 150 
IMAGE_HEIGHT = 150 

# 修改测试图片地址
image_paths = 'test/cat.681.jpg'
#image_paths = 'test/dog.592.jpg'
model_server_url = 'http://127.0.0.1:8501/v1/models/sagemaker-demo:predict'

In [4]:
from keras.preprocessing import image

images = image.load_img(image_paths, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
input_image = image.img_to_array(images)
input_image = np.expand_dims(input_image, axis=0)
input_image /= 255.

input_images = input_image.tolist()
data = json.dumps({"inputs":input_images})

headers = {"content-type": "application/json"}


json_response = requests.post(model_server_url, data=data, headers=headers)
json_result = json.loads(json_response.text)
print(json_result)

Using TensorFlow backend.


{'outputs': [[0.996342242, 0.00365771]]}


In [ ]:
# 根据需要修改class name，按字典序
class_name=['cat','dog']
class_name[np.argmax(json_result["outputs"])]

## 推送docker镜像到ECR

In [ ]:
%%sh
algorithm_name=sagemaker-cat-vs-dog
REGION=$(aws configure get region)
ACCOUNT=$(aws sts get-caller-identity --query Account --output text)

# If the ECS repository doesn't exist, creates it.
aws ecr create-repository --repository-name ${algorithm_name} > /dev/null 2>&1

# ECR requires the image name to be in this format:
REPOSITORY_NAME=${ACCOUNT}.dkr.ecr.${REGION}.amazonaws.com.cn/${algorithm_name}:latest

# Tags the image with the expect ECR format
docker tag sagemaker-cat-vs-dog ${REPOSITORY_NAME}

# Allows docker access to ECR
$(aws ecr get-login --no-include-email)

# pushes the image to ECR
docker push ${REPOSITORY_NAME}

## 部署模型到SageMaker

In [7]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
ecr_repository = 'sagemaker-cat-vs-dog'
tag = ':latest'
uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'
repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)
print(repository_uri)

315505707008.dkr.ecr.cn-northwest-1.amazonaws.com.cn/sagemaker-cat-vs-dog:latest


In [ ]:
from sagemaker.model import Model
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
#role="arn:aws-cn:iam::315505707008:role/service-role/AmazonSageMaker-ExecutionRole-20200430T124235" 
image_uri = repository_uri
endpoint_name = "sagemaker-cat-vs-dog"
my_model = Model(
            role=role,
            image_uri=image_uri)

#该步骤大概需要10分钟
xgb_predictor = my_model.deploy(initial_instance_count=1,
                                endpoint_name=endpoint_name,
                                instance_type='ml.t2.medium'
                                )

## 推理
### 读取数据

In [5]:
from keras.preprocessing import image
import json
import numpy as np

IMAGE_WIDTH = 150 
IMAGE_HEIGHT = 150 
# 修改测试图片地址
image_paths = 'test/cat.681.jpg'
#image_paths = 'test/dog.592.jpg'
images = image.load_img(image_paths, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
input_image = image.img_to_array(images)
input_image = np.expand_dims(input_image, axis=0)
input_image /= 255.

input_images = input_image.tolist()

data = json.dumps({"name": 'tensorflow/serving/predict',"signature_name":'predict',"inputs":input_images})

In [ ]:
input_image.shape

### 方式一、使用boto3

In [ ]:
import boto3
endpoint_name = "sagemaker-cat-vs-dog"
client = boto3.client('runtime.sagemaker')

response = client.invoke_endpoint(EndpointName=endpoint_name,
                                  Body=data)
print(response)

In [ ]:
response_body = response['Body'] 
body= response_body.read()
results = json.loads(body.decode('utf-8'))
print(results)

### 方式二、使用SageMaker SDK

In [ ]:
import sagemaker
xgb_predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker.Session())

In [ ]:
result = xgb_predictor.predict(data)
results = json.loads(result.decode('utf-8'))
print(results)

## 删除Endpoint

In [ ]:
import boto3
sage = boto3.Session().client(service_name='sagemaker') 
sage.delete_endpoint(EndpointName=endpoint_name)